In [1]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from fastai.text.all import *
from fastai import *
from blurr import *
#from transformers import *
import transformers
from blurr.text.data.all import *
from blurr.text.modeling.all import *

import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from fastbook import load_learner
import boto3

C:\Users\Yao\Anaconda3\lib\site-packages\blurr\text\modeling\question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Yao\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [59]:
##!pip install fastbook
##!pip install fastai
#!pip install transformers
##!pip install blurr
#!pip install --upgrade h5py
#!pip install libhdf5
#print(h5py.__version__)
#!pip install bert_score

In [27]:
#!pip install transformers -Uqq
#!pip install datasets -Uqq
#!pip install bert-score -Uqq
#!pip install sacremoses 
#!pip install ohmeow-blurr -Uqq

In [2]:
df_test = pd.read_csv('dataset-test.csv') # use test file just to load faster
df_test = df_test[df_test['subreddit'] == 'leagueoflegends'] #subset
test_texts = df_test[['content','summary']]

In [ ]:
#df_train = pd.read_csv('dataset-train.csv')
#train_texts = df_train[['content','summary']]

In [82]:
def pre_processing(data):
    stop_words = set(stopwords.words('english')) #use stop words
    #data = re.sub('(\d+)', '', data) #remove digits
    data = re.sub('\W+',' ', data) #remove special chaacters
    word_tokens = word_tokenize(data.lower()) #tokenize the string
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words] #remove stop words
    #porter_stemmer = PorterStemmer() #stem the words 
    #stemmed_words = [porter_stemmer.stem(word) for word in filtered_sentence]
    wordnet_lemmatizer = WordNetLemmatizer() #lemm the word, does better than stem the word
    lemm_words = [wordnet_lemmatizer.lemmatize(word) for word in filtered_sentence]
    return ' '.join(lemm_words)

In [26]:
def f_rscore(reference, candidate): #assuming both are lists, Rouge
    rscores = []
    rscorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    for i in range(len(reference)):
        rscore = rscorer.score(reference[i], candidate[i])
        rscores.append(list(rscore.values())[0][2]) # fscore
    return rscores

In [4]:
pretrained_model_name = "facebook/bart-large-cnn"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=transformers.BartForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='content',
    target_text_attr='summary',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(test_texts) #test_texts here


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('content'), get_y=ColReader('summary'), splitter=RandomSplitter())
# Batch size can be changed here
dls = dblock.dataloaders(preprocessed_train, bs = 8)

C:\Users\Yao\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


C:\Users\Yao\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


In [7]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()

In [19]:
learn.save('reddit_summary.pkl')

Path('models/reddit_summary.pkl.pth')

In [20]:
learner = learn.load('reddit_summary.pkl')

In [21]:
#inf_learn = load_learner('Reddit-Title-Generation/notebooks/reddit_summary.pkl')

In [36]:
outputs = learn.blurr_generate(test_texts.iloc[0].content, early_stopping=False, num_return_sequences=1, \
                               min_length=30, max_length=50)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': 'Riot needs to stop trying to please all the new players begging for legacy content and start to honor the old players that have been investing in the game from the very beginning. If Riot puts them up for sale again they lose all value'}



$$prediction$$

In [90]:
df_test = pd.read_csv('dataset-test.csv')

In [96]:
df_a = df_test[df_test['subreddit'] == 'AskReddit'].sample(50)[['content','summary']]
df_l = df_test[df_test['subreddit'] == 'leagueoflegends'].sample(50)[['content','summary']]
df_t = df_test[df_test['subreddit'] == 'tifu'].sample(50)[['content','summary']]
df_r = df_test[df_test['subreddit'] == 'relationships'].sample(50)[['content','summary']]
df_ra = df_test[df_test['subreddit'] == 'relationship_advice'].sample(50)[['content','summary']]

In [85]:
#preprocessing optional
df_a['content'] = list(map(pre_processing, df_a['content'])) 
df_a['summary'] = list(map(pre_processing, df_a['summary'])) 
df_l['content'] = list(map(pre_processing, df_l['content'])) 
df_l['summary'] = list(map(pre_processing, df_l['summary'])) 
df_t['content'] = list(map(pre_processing, df_t['content'])) 
df_t['summary'] = list(map(pre_processing, df_t['summary'])) 
df_r['content'] = list(map(pre_processing, df_r['content'])) 
df_r['summary'] = list(map(pre_processing, df_r['summary'])) 
df_ra['content'] = list(map(pre_processing, df_ra['content'])) 
df_ra['summary'] = list(map(pre_processing, df_ra['summary'])) 

In [97]:
df_a.iloc[0].content

"I dunno, that's the way I feel. I wouldn't want to get more involved than I already was. And if the same situation was occurring with my wife - I'd expect the same. You're right, karma. Im not particularly one to be a bystander. In this case i would be. Liars get caught and once a cheater always a cheater. If it'd make you feel better - send her a letter. I don't think my sanity could handle bearing that news upon someone. But, after hearing  what the fuck is wrong with you  it's clear I'm in the minority. Thanks for the colorful language. I'm sorry you feel so strongly about my opinion. On a different note I can recall the triple-homicide that occurred in Vermont. It was a love triangle that ended with a hunting rifle. I tried my best to locate the article on google. I'm afraid it occurred in the late 90's so it might not have a net report. It was very upsetting for the community."

In [100]:
def make_predictions(test_texts):
    #list(outputs[0].values())
    predictions = []
    for i in range(len(test_texts)):
        outputs = learn.blurr_generate(test_texts.iloc[i].content, early_stopping=False, num_return_sequences=1, \
                                   min_length=30, max_length=50)
        predictions.append(list(outputs[0].values())[0])
    return predictions

In [104]:
a_p = make_predictions(df_a)
l_p = make_predictions(df_l)
t_p = make_predictions(df_t)
r_p = make_predictions(df_r)
ra_p = make_predictions(df_ra)

In [102]:
def avg_rouge(test_texts, predictions):
    score_list = f_rscore(list(test_texts.summary), predictions)
    return np.average(score_list)

In [103]:
avg_rouge(df_a, a_p)

0.11916190621829653

In [105]:
avg_rouge(df_l, l_p)

0.11731240898370848

In [106]:
avg_rouge(df_t, t_p)

0.14023053352398496

In [107]:
avg_rouge(df_r, r_p)

0.16361195193043446

In [108]:
avg_rouge(df_ra, ra_p)

0.15174874941464958

In [115]:
score_list = f_rscore(list(df_a.summary), a_p)
[(i ,e) for i, e in enumerate(score_list) if e < 0.05]

[(2, 0.0),
 (5, 0.0),
 (6, 0.04166666666666667),
 (16, 0.041666666666666664),
 (19, 0.0),
 (21, 0.0),
 (23, 0.04545454545454545),
 (26, 0.0),
 (28, 0.038461538461538464),
 (32, 0.03636363636363636),
 (33, 0.04347826086956522),
 (36, 0.0),
 (37, 0.04),
 (47, 0.04)]

In [125]:
score_list = f_rscore(list(df_a.summary), a_p)
[(i ,e) for i, e in enumerate(score_list) if e >0.2]

[(1, 0.5217391304347825),
 (14, 0.23809523809523808),
 (24, 0.25),
 (39, 0.2285714285714286),
 (41, 0.2156862745098039),
 (45, 0.2181818181818182)]

$$Example$$

In [117]:
df_a.iloc[2].content

'Not me, but a friend. \n She\'s an animator. She\'s actually quite good. She got a booth at an anime convention to sell some things. We helped her by manning the booth and getting people to buy stuff. \n She opened and manned the booth the entire first day, we would visit her to see how it was going, not to good. \n The second day she texted us and said "I figured out how to sell!"\nWhen I showed up to man the booth for my shift, there was barely anything left. She was wearing a bikini top and really short shorts.'

In [118]:
df_a.iloc[2].summary

'Anything will be sold if there are boobs involved.'

In [119]:
a_p[2]

'A friend got a booth at an anime convention to sell some things. We helped her by manning the booth and getting people to buy stuff. The second day she texted us and said "I figured out how to sell!"'

$$Example$$

In [120]:
df_a.iloc[5].content

'A most excellent'

$$Example$$

In [121]:
df_a.iloc[19].content

'This story applies to your situation fairly well. \n A few years ago, I started dating this girl I met in college. She was a few years younger than me. She was beautiful, long blonde hair, sexy smile and had that confidence that made her all the more attractive. \n We started dating early in the first semester and became fairly serious pretty quickly. By the end of the school year, we had decided we would move into together for the summer and the following year, this was becoming a fairly serious relationship. I was very excited to take these next steps with her because she was unlike any other lady I had dated prior. \n Well when we moved in together, she stopped putting any effort into herself at all. She stopped combing her hair, started wearing sweat pants and hoodies every day and generally just looked disheveled all the time. \n I still loved her though because I loved her for who she was and not what she wore or looked like. \n That was up until the day she stopped wearing make

In [122]:
df_a.iloc[19].summary

'Sometimes makeup is essential in a relationship.'

In [123]:
a_p[19]

'"I loved her for who she was and not what she wore or looked like" "I noticed she was 500 feet tall and from the paleolithic era" "Goddamn lochness monster, I ain\'t giving you no'

$$Example$$

In [129]:
df_a.iloc[24].content

'My best friend teaches freshman english. After class one day, a note was left and a group of girls in the next class grabbed it, read it, and started giggling. My friend grabbed the note from the girls, and threw it away without reading it, trying to save the student (a quite and polite student), and probably herself, some humiliation. However, when the next day rolled around and the same girl left another note behind, it was fair game. The 14-year-old had left behind a note addressed to her boyfriend and it read: "Hey, wanna come over after school for some sex and capri sun?". Unsure of what Capri Sun had to do with sex, my friend decided to search for it on urbandictionary.com. "Capri Sun" means getting jizzed on the face after a blow job. She was never able to look at that girl the same.'

In [130]:
df_a.iloc[24].summary

'14-year old freshman wanted her boyfriend to come on her face.'

In [131]:
a_p[24]

'"Capri Sun" means getting jizzed on the face after a blow job. The 14-year-old had left behind a note addressed to her boyfriend and it read: "Hey, wanna come over after school for some'